# Group7
## American Express Project
### Group members:
* ME19B005
* ME19B028
* ME19B084
* ME19B110
* ME19B127
* ME19B130

# Importing libraries

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV

# Reading Train and Test data

In [20]:
train_data=pd.read_csv("Training Data_2021.csv")
train_data.head()

c:\users\nivash\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,default_ind
0,230032,1696,1.6541,0.000,0.0,0.0,0,6015,322,40369,...,1,73.78,82.547,0.08696,10,0.63899,na,0,C,0
1,230033,1846,0.8095,0.000,0.0,0.0,102,7532,3171,18234,...,0,99.129,missing,0,13,0.63836,na,na,L,1
2,230034,1745,0.4001,0.000,0.0,0.0,missing,2536,missing,missing,...,0,missing,29.29,0,1,1.00000,na,0,C,1
3,230035,1739,0.2193,0.000,0.0,0.0,1982,26440,4955,20316,...,0,96.272,missing,0.15385,3,0.53241,0,0,L,0
4,230036,1787,0.0118,0.225,0.0,0.0,5451,5494,5494,7987,...,0,115.019,missing,0,1,0.92665,na,na,L,0


In [21]:
test_data=pd.read_csv("Test Data_2021.csv")
test_data.head()

c:\users\nivash\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47
0,578069,1719,0.6174,8.623,0.0,0.0,258,258,258,10729,...,2,0,missing,missing,0.54545,2,0.91837,0,0,C
1,578070,1795,0.2051,0.000,0.0,0.0,1685,12711,8913,80519,...,4,0,missing,missing,0.17241,4,0.94563,0,0,C
2,578071,1742,0.5082,0.000,0.0,0.0,1185,8954,8954,1189,...,1,0,missing,missing,0.64706,1,0.97054,0,0,C
3,578072,1685,0.2595,25.409,0.0,0.0,missing,3354,missing,missing,...,2,0,missing,missing,0.85714,1,1.00000,na,0,C
4,578073,1666,1.2678,0.000,0.0,0.0,570,570,570,missing,...,2,0,missing,101.61,missing,0,0.99617,na,0,L


In [22]:
print("Columns:",train_data.columns)
print("Shape:",train_data.shape)
print("Zeros:",train_data[train_data["default_ind"]==0]["default_ind"].count())
print("Ones:",train_data[train_data["default_ind"]==1]["default_ind"].count())

Columns: Index(['application_key', 'mvar1', 'mvar2', 'mvar3', 'mvar4', 'mvar5', 'mvar6',
       'mvar7', 'mvar8', 'mvar9', 'mvar10', 'mvar11', 'mvar12', 'mvar13',
       'mvar14', 'mvar15', 'mvar16', 'mvar17', 'mvar18', 'mvar19', 'mvar20',
       'mvar21', 'mvar22', 'mvar23', 'mvar24', 'mvar25', 'mvar26', 'mvar27',
       'mvar28', 'mvar29', 'mvar30', 'mvar31', 'mvar32', 'mvar33', 'mvar34',
       'mvar35', 'mvar36', 'mvar37', 'mvar38', 'mvar39', 'mvar40', 'mvar41',
       'mvar42', 'mvar43', 'mvar44', 'mvar45', 'mvar46', 'mvar47',
       'default_ind'],
      dtype='object')
Shape: (83000, 49)
Zeros: 59145
Ones: 23855


# Data Cleaning
## Replace all missing value to Nan

In [23]:
train_data.replace({'na':np.NaN,'missing':np.NaN},inplace=True) 
test_data.replace({'na':np.NaN,'missing':np.NaN},inplace=True)

# Changing categorical column to binary
train_data["mvar47"] = (train_data["mvar47"] == "C").astype(int)
test_data["mvar47"] = (test_data["mvar47"] == "C").astype(int)
train_data = train_data.astype(float)
test_data = test_data.astype(float)

In [24]:
x_Train=train_data.iloc[:,1:-1]# Spliting independent variable
y_Train=train_data.iloc[:,-1]# Spliting Dependent variable
test_index=test_data["application_key"].astype(int)# Getting Index for final submission
test_data=test_data.iloc[:,1:]

## Dropping columns having many missing values

In [25]:
#Checking how many values missing in each columns
empty=[] # Number of missing values in each column
percent=[] # Percentage of missing value in each column
emt_clmn=[] # Column names that have missing value greater than threshold value
for i in range(1,47):
    txt="mvar{}".format(i)
    empty.append(x_Train[x_Train[txt].isna()].shape[0])
    percent.append(empty[i-1]/830)
    if percent[i-1]>=56: # Threshold value is 56, finialized by trial and error
        emt_clmn.append(txt)
print(emt_clmn)

# Drop the columns in emt_clean
for i in emt_clmn:
    x_Train.drop(i,axis=1,inplace = True)
    test_data.drop(i,axis=1,inplace = True)
print("Shape",x_Train.shape)

['mvar31', 'mvar40', 'mvar41']
Shape (83000, 44)


## Dropping highly correlated columns

In [26]:
#Checking for highly correlated columns
correlation=train_data.corr()
I=[]
J=[]
t=0.9 # Threshold for high correlation, finalized by trial and error
for i in range(1,39):
    for j in range(1,39):
        if i>j:
            if correlation.iloc[i][j] > t or correlation.iloc[i][j] < -t:
                I.append(i)
                J.append(j)
Index = []
for i in range(len(I)):
        Index.append([correlation.columns[I[i]],correlation.columns[J[i]]])
Index

[['mvar17', 'mvar16'],
 ['mvar18', 'mvar16'],
 ['mvar18', 'mvar17'],
 ['mvar20', 'mvar19'],
 ['mvar27', 'mvar26']]

In [27]:
# Removing highly correlated columns
x_Train.drop(["mvar17","mvar16","mvar19","mvar26"],axis=1,inplace=True) 
test_data.drop(["mvar17","mvar16","mvar19","mvar26"],axis=1,inplace=True)

## Handling skewness in the data

In [31]:
# Logarithmic transformation
x_Train = np.log(x_Train+1)
test_data = np.log(test_data+1)

# Machine Learning

In [32]:
# Seperating rows having 0s and 1s for handling imbalance in the dataset
train = pd.concat([x_Train,y_Train],axis=1)
train_0 = train[train["default_ind"]==0]
train_1 = train[train["default_ind"]==1]

## Ensembe

In [33]:
# This function produces balanced batches of data for training 
def get_train_batch(train_0,train_1,rand):
    df_train = pd.concat([train_0.sample(train_1.shape[0],random_state=rand),train_1],axis=0)
    
    x_train = df_train.drop("default_ind",axis = 1)
    y_train = df_train["default_ind"]
    
    return x_train,y_train

In [34]:
xgb=XGBClassifier(max_depth=6) # Tried various algorithms and finalized xgboost based on its performance, parameters are tuned to give higher f1_score
model = CalibratedClassifierCV(xgb, method='sigmoid', cv=2) # Calibrating the model with 2 fold cross validation
k=31 # Total number of model batches, fixed after various trials
y_test=[]
for i in range(k): # Ensemble loop
    X_train,Y_train = get_train_batch(train_0,train_1,i)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    Test_data = scaler.transform(test_data)
    model.fit(X_train,Y_train) 
    y_test.append(model.predict(Test_data))

# Final prediction based on majority vote
y_final_test=[]
for i in range(len(y_test[1])):
    n=0
    for j in range(k):
        n=n+y_test[j][i]
    if n>15:
        y_final_test.append(1)
    else:
        y_final_test.append(0)

c:\users\nivash\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:14:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\nivash\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:14:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\nivash\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[07:25:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\nivash\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:25:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\nivash\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:25:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [36]:
y_test=pd.DataFrame(y_final_test,index=test_index) # Indexing for submission
y_test.to_csv("Group7_26.csv") # Writing final csv file 